In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
## 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
## 天津市
roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
## 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
## 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
## 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)


In [4]:
# 定义参数
# region = 'daqing'
# sample_region = 'DAQING'
region = 'tianjin'
sample_region = 'TIANJIN'
# region = 'wuhan'
# sample_region = 'WUHAN'
# region = 'prd'
# sample_region = 'PRD'
# region = 'tibet'
# sample_region = 'TIBET'

In [5]:
### 验证函数
water_image = ee.Image.constant(1).toFloat().rename('label')
land_image = ee.Image.constant(0).toFloat().rename('label')
def verify(img_id):
    auto_point = auto_samples.filter(ee.Filter.eq('Image_id',img_id))
    man_point = man_samples.filter(ee.Filter.eq('Image_id',img_id))
    water = water_image.clipToCollection(man_point.filter(ee.Filter.eq('label',1)))
    land = land_image.clipToCollection(man_point.filter(ee.Filter.eq('label',0)))
    image_waterclass = ee.ImageCollection([water,land]).sum()
    inter_points = image_waterclass.sampleRegions(**{
        'collection': auto_point,
        'properties': ['waterclass','Image_id','point_type'],
        'scale': 30,
        'geometries': True,
    })
    return inter_points

def removeProperty(feat):
    selectProperties = ee.Feature(feat).propertyNames().filter(ee.Filter.neq('item', 'random')).filter(ee.Filter.neq('item', 'waterclass'))  
    return feat.select(selectProperties)

In [6]:
# TIME = [['1999','1999'],['2004','2004'],['2009','2009'],['2014','2014'],['2019','2019']]
TIME = ['1999','2004','2009','2014','2019']
# TIME = ['2019']
for time in TIME:
    print(time)
    # 人工选取的样本点
    man_samples = ee.FeatureCollection('users/311605001111/validation_samples_of_aut-datasets/' + region + '_' + time)
    # print('manual-sample number:{}'.format(man_samples.size().getInfo()))
    auto_samples = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + time + '_awei')
    # print('auto-sample number:{}'.format(auto_samples.size().getInfo()))
    ### 唯一ID
    manual_samples = man_samples.distinct('Image_id')
    automatic_samples = auto_samples.distinct('Image_id')
    filter = ee.Filter.equals(**{'leftField': 'Image_id','rightField': 'Image_id'})
    id_col = ee.List(ee.Join.simple().apply(automatic_samples, manual_samples, filter).aggregate_array('Image_id'))
    # print(id_col.size().getInfo())
    SamplePoints = ee.FeatureCollection(id_col.map(verify)).flatten()

    permanent_water = SamplePoints.filter(ee.Filter.eq('point_type','permanent_water')).randomColumn('random',1,'uniform').sort('random').limit(25)
    season_water = SamplePoints.filter(ee.Filter.eq('point_type','season')).randomColumn('random',1,'uniform').sort('random').limit(25)
    no_water = SamplePoints.filter(ee.Filter.eq('point_type','no_water')).randomColumn('random',1,'uniform').sort('random').limit(150)
    com_SamplePoints = ee.FeatureCollection([permanent_water,season_water,no_water]).flatten()
    
    test_accuracy = SamplePoints.errorMatrix('label', 'waterclass')
    print(test_accuracy.getInfo())
    print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
    print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

    test_accuracy = com_SamplePoints.errorMatrix('label', 'waterclass')
    print(test_accuracy.getInfo())
    print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
    print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

1999
[[185, 4], [0, 64]]
total accuracy:0.9841897233201581
kappa:0.9590150656082941
[[158, 3], [0, 39]]
total accuracy:0.985
kappa:0.9535747446610956
2004
[[207, 2], [0, 93]]
total accuracy:0.9933774834437086
kappa:0.9845547997749705
[[150, 2], [0, 48]]
total accuracy:0.99
kappa:0.9729729729729729
2009
[[214, 0], [2, 77]]
total accuracy:0.9931740614334471
kappa:0.9825293661677896
[[160, 0], [1, 39]]
total accuracy:0.995
kappa:0.9842271293375394
2014
[[244, 1], [4, 75]]
total accuracy:0.9845679012345679
kappa:0.9576049408562757
[[164, 1], [3, 32]]
total accuracy:0.98
kappa:0.929140832595217
2019
[[203, 1], [1, 71]]
total accuracy:0.9927536231884058
kappa:0.9812091503267973
[[165, 0], [0, 35]]
total accuracy:1
kappa:1


In [ ]:
# TIME = [['1999','1999'],['2004','2004'],['2009','2009'],['2014','2014'],['2019','2019']]
# TIME = ['1999','2004','2009','2014','2019']
TIME = ['2019']
for time in TIME:
    print(time)
    # 人工选取的样本点
    man_samples = ee.FeatureCollection('users/311605001111/validation_samples_of_aut-datasets/' + region + '_' + time)
    print('manual-sample number:{}'.format(man_samples.size().getInfo()))
    auto_samples = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + time + '_awei')
    # auto_samples = samples.map(lambda i :i.set({'year': ee.String(i.get('Image_id')).slice(-8,-4)})).filter(ee.Filter.eq('year',time[0]))
    print('auto-sample number:{}'.format(auto_samples.size().getInfo()))
    ### 唯一ID
    manual_samples = man_samples.distinct('Image_id')
    automatic_samples = auto_samples.distinct('Image_id')
    filter = ee.Filter.equals(**{'leftField': 'Image_id','rightField': 'Image_id'})
    id_col = ee.List(ee.Join.simple().apply(automatic_samples, manual_samples, filter).aggregate_array('Image_id'))
    print(id_col.size().getInfo())
    SamplePoints = ee.FeatureCollection(id_col.map(verify)).flatten()

    permanent_water = SamplePoints.filter(ee.Filter.eq('point_type','permanent_water')).randomColumn('random',1,'uniform').sort('random').limit(25)
    season_water = SamplePoints.filter(ee.Filter.eq('point_type','season')).randomColumn('random',1,'uniform').sort('random').limit(25)
    no_water = SamplePoints.filter(ee.Filter.eq('point_type','no_water')).randomColumn('random',1,'uniform').sort('random').limit(150)
    com_SamplePoints = ee.FeatureCollection([permanent_water,season_water,no_water]).flatten()
    print(com_SamplePoints.size().getInfo())
    test_accuracy = com_SamplePoints.errorMatrix('label', 'waterclass')
    print(test_accuracy.getInfo())
    print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
    print('kappa:{}'.format(test_accuracy.kappa().getInfo()))
#     dataset_id = sample_region + time
#     assetID = 'users/311605001111/man_samples_200' + '/' + dataset_id
#     task = ee.batch.Export.table.toAsset(**{
#         'collection': com_SamplePoints.map(removeProperty),
#         'description': dataset_id,
#         'assetId': assetID
#     })
#     task.start()

# 查询异常点分布

In [ ]:
region = 'prd'
sample_region = 'PRD'
time = '2014'
# 核心代码
###数据
man_samples = ee.FeatureCollection('users/311605001111/validation_samples_of_aut-datasets/' + region + '_' + time)
print('manual-sample number:{}'.format(man_samples.size().getInfo()))
auto_samples = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + time)
print('auto-sample number:{}'.format(auto_samples.size().getInfo()))
### 唯一ID
manual_samples = man_samples.distinct('Image_id')
automatic_samples = auto_samples.distinct('Image_id')
filter = ee.Filter.equals(**{'leftField': 'Image_id','rightField': 'Image_id'})
id_col = ee.List(ee.Join.simple().apply(automatic_samples, manual_samples, filter).aggregate_array('Image_id'))
print(id_col.size().getInfo())
print(id_col.getInfo())

In [ ]:
### 验证函数
water_image = ee.Image.constant(1).toFloat().rename('label')
land_image = ee.Image.constant(0).toFloat().rename('label')
def verify(img_id):
    auto_point = auto_samples.filter(ee.Filter.eq('Image_id',img_id))
    man_point = man_samples.filter(ee.Filter.eq('Image_id',img_id))
    water = water_image.clipToCollection(man_point.filter(ee.Filter.eq('label',1)))
    land = land_image.clipToCollection(man_point.filter(ee.Filter.eq('label',0)))
    image_waterclass = ee.ImageCollection([water,land]).sum()
    inter_points = image_waterclass.sampleRegions(**{
        'collection': auto_point,
        'properties': ['waterclass','Image_id','point_type'],
        'scale': 30,
        'geometries': True,
    })
    return inter_points

### 精度验证
SamplePoints = ee.FeatureCollection(id_col.map(verify)).flatten()
print(SamplePoints.size().getInfo())

In [ ]:
permanent_water = SamplePoints.filter(ee.Filter.eq('point_type','permanent_water')).randomColumn('random',1,'uniform').sort('random').limit(25)
season_water = SamplePoints.filter(ee.Filter.eq('point_type','season')).randomColumn('random',1,'uniform').sort('random').limit(25)
no_water = SamplePoints.filter(ee.Filter.eq('point_type','no_water')).randomColumn('random',1,'uniform').sort('random').limit(150)
com_SamplePoints = ee.FeatureCollection([permanent_water,season_water,no_water]).flatten()
print(com_SamplePoints.size().getInfo())

test_accuracy = SamplePoints.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

test_accuracy = com_SamplePoints.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

In [ ]:
for i in id_col.getInfo():
    print(i)
    diff_water = SamplePoints.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',0))
    diff_nowater = SamplePoints.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1))
    all_outlier = ee.FeatureCollection([diff_water,diff_nowater]).flatten()
    outlier = all_outlier.filter(ee.Filter.eq('Image_id',i))
    print(outlier.size().getInfo())
    

In [ ]:
diff_water = SamplePoints.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',0))
diff_nowater = SamplePoints.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1))
all_outlier = ee.FeatureCollection([diff_water,diff_nowater]).flatten()
outlier = all_outlier.filter(ee.Filter.eq('Image_id','LANDSAT/LE07/C01/T1_SR/LE07_122045_20090907'))
print(outlier.size().getInfo())
print(outlier.getInfo())

# 验证点，空间分布

In [ ]:
elevation = ee.Image("users/311605001111/hillshade_" + region)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

# 图像可视化参数
visParams = {'bands': ['B4', 'B3', 'B2'],'min': 0,'max': 3000,'gamma': 1.4}

# l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
#        .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
#        .filterBounds(roi) \
#        .filter(ee.Filter.lt('CLOUD_COVER',30)) \
#        .filterDate('2019-01-01', '2019-12-31')
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate('2019-01-01', '2019-12-31')
image = l8.map(maskSR).median().clip(roi)
Map.addLayer(image, visParams,'image mosaic')
print(l8.size().getInfo())

In [ ]:
# region = 'daqing'
# sample_region = 'DAQING'
# region = 'tianjin'
# sample_region = 'TIANJIN'
# region = 'wuhan'
# sample_region = 'WUHAN'
region = 'prd'
sample_region = 'PRD'
# region = 'tibet'
# sample_region = 'TIBET'

In [ ]:
# 合成影像
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
# # 大庆市
# re = [125.3055, 45.8362, 123.9355, 46.8762]
# # 天津市
# re = [117.4944, 38.6272, 115.9244, 39.6372]
# # 武汉市
# re = [115.0993, 29.8642, 113.7393, 30.9242]
# # 珠三角
re = [114.0514, 22.2347, 112.7614, 23.2547]
# 青藏高原
# re = [91.3626, 34.6579, 89.8826, 35.8279]

fig = plt.figure(figsize=(12, 8))
refer_img = ee.Image('users/311605001111/basemap_image/' + region).visualize(**{'opacity':0.85})
manual_samples = ee.FeatureCollection('users/311605001111/man_samples_200/' + sample_region + '2019')
pwater = manual_samples.filter(ee.Filter.eq('point_type','permanent_water')).draw('blue')
nwater = manual_samples.filter(ee.Filter.eq('point_type','no_water')).draw('red')
swater = manual_samples.filter(ee.Filter.eq('point_type','season')).filter(ee.Filter.eq('label',1)).draw('blue')
snowater = manual_samples.filter(ee.Filter.eq('point_type','season')).filter(ee.Filter.eq('label',0)).draw('red')
blend = refer_img.blend(pwater).blend(nwater).blend(swater).blend(snowater)
Map.addLayer(blend, {}, "Blend")
# use cartoee to get a map
ax = cartoee.get_map(blend, region=re)
cartoee.add_gridlines(ax, interval=[0.4,0.4], linestyle=":", ytick_rotation="vertical")
ax.set_title(label = '(d)      Region D', fontsize=28)

In [ ]:
# 合成影像
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
# # 大庆市
# re = [123.9355, 45.8362,125.3055, 46.8762]
# # 天津市
# re = [115.9244, 38.6272,117.4944,39.6372]
# # 武汉市
# re = [113.7393, 29.8642,115.0993, 30.9242]
# # 珠三角
# re = [112.7614, 22.2347,114.0514, 23.2547]
# 青藏高原
re = [89.8826, 34.6579,91.3626, 35.8279]
fig = plt.figure(figsize=(12, 8))
refer_img = ee.Image('users/311605001111/basemap_image/' + region).visualize(**{'opacity':0.85})
manual_samples = ee.FeatureCollection('users/311605001111/man_samples_200/' + sample_region + '2019')
pwater = manual_samples.filter(ee.Filter.eq('point_type','permanent_water')).draw('blue')
nwater = manual_samples.filter(ee.Filter.eq('point_type','no_water')).draw('red')
swater = manual_samples.filter(ee.Filter.eq('point_type','season')).draw('yellow')
blend = refer_img.blend(pwater).blend(nwater).blend(swater)
Map.addLayer(blend, {}, "Blend")
# use cartoee to get a map
ax = cartoee.get_map(blend, region=re)
cartoee.add_gridlines(ax, interval=[0.4,0.4], linestyle=":", ytick_rotation="vertical")

ax.set_title(label = 'Region E', fontsize=28)